In [9]:
# Import
import mysql.connector  
import time

# Import
import sys
sys.path.append('../')
 
from a_models.jobdesc_cleanser import *   

entity = "jobdescription"

# is_insert   = input(" is_insert ?")  
is_insert   = "yes"
# ----------------------------------------------------------   
space = "__"
# ----------------------------------------------------------   
host="localhost"
user="root" 
database="pr_scraping_job_vacancy_indonesia_2023_07"
mydb = mysql.connector.connect(host=host,user=user,password="",database=database)
# ----------------------------------------------------------   
query = " Select "
query += " fk_id  "  
query += " from " + entity    
query += " order by fk_id desc "   
query += " limit 1 "    
# ----------------------------------------------------------    
mycursor = mydb.cursor()
mycursor.execute(query) 
result_check = mycursor.fetchone()
# ---------------------------------------------------------- 
mycursor.close()
mydb.close()
# ----------------------------------------------------------  
if result_check is not None and len(result_check) > 0:
    last_id = result_check[0]
else: 
    last_id = 0
# ---------------------------------------------------------- 
print("Last ID:" + str(last_id), flush=True) 
# ---------------------------------------------------------- 
  
# ----------------------------------------------------------   
query = " Select "
query += " scriptx  "  
query += " , id  "  
query += " from jobstreet_raws_2023_07 "     
query += " where id > " + str(last_id)   
query += " order by id asc "     
query += " limit 5000 "     
# ----------------------------------------------------------  
# print(space + query, flush=True)
# ----------------------------------------------------------    
mydb = mysql.connector.connect(host=host,user=user,password="",database=database)
mycursor = mydb.cursor()
mycursor.execute(query)
myresult = mycursor.fetchall()
mycursor.close()
mydb.close()
# ----------------------------------------------------------
counter_row = 1
# ----------------------------------------------------------
for x in myresult:    
    # ------------------------------------------------------ 
    time.sleep(0.005)
    # ------------------------------------------------------ 
    scriptx     = x[0] 
    idx         = x[1] 
    # ------------------------------------------------------ 
    pre_scriptx     = scriptx.split('"jobDetail"')   
    # ------------------------------------------------------ 
    pre_scriptx2    = pre_scriptx[1].split('"jobDescription":{')   
    # ------------------------------------------------------   
    pre_scriptx3    = pre_scriptx2[1].split('}')   
    # ------------------------------------------------------ 
    text            = pre_scriptx3[0] 
    # ------------------------------------------------------ 
    # print(str(idx) + " __ " + text, flush=True) 
    # ------------------------------------------------------ 
    text_clean = clean_please(text)
    # ------------------------------------------------------ 
    # ------------------------------------------------------ 
    # print(text_clean, flush=True) 
    # ------------------------------------------------------ 
    # ------------------------------------------------------ 
    split_clean = text_clean.split("|") 
    # ------------------------------------------------------  
    # ------------------------------------------------------  
    if(is_insert == "yes"):
        # --------------------------------------------------  
        for sc in split_clean:
            # ----------------------------------------------  
            time.sleep(0.005)
            # ----------------------------------------------  
            if(sc != ""):
                # ------------------------------------------  
                jumlah_spasi    = sc.count(" ") 
                # ------------------------------------------  
                label_o         = ("O " * (jumlah_spasi + 1)).strip() 
                # ------------------------------------------  
                query_commit = "INSERT INTO `"+entity+"`( "
                # ------------------------------------------  
                query_commit += " `fk_id`, " 
                query_commit += " `name`, " 
                query_commit += " `label`, " 
                query_commit += " `length` " 
                query_commit += " ) VALUES ( "
                query_commit += " '" + str(idx) + "', "    
                query_commit += " '" + str(sc).replace("'", "\\'") + "', "
                query_commit += " '" + str(label_o) + "', "    
                query_commit += " '" + str(len(sc)) + "' "    
                query_commit += " )  " 
                # ------------------------------------------   
                if(counter_row == 500):
                    print(space + query_commit, flush=True)
                    counter_row = 1
                # ------------------------------------------  
                mydb = mysql.connector.connect(host=host,user=user,password="",database=database)
                mycursor = mydb.cursor()
                mycursor.execute(query_commit)
                mydb.commit()     
                # ------------------------------------------  
                mycursor.close()
                mydb.close()
                # ------------------------------------------   
                counter_row += 1
                # ------------------------------------------   
                # ------------------------------------------  
                # ------------------------------------------  
            # ----------------------------------------------  
        # --------------------------------------------------  
    # ------------------------------------------------------ 
print("end")

Last ID:48673
end
